<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/logo.png?raw=true'/></center>

<h1 align = 'center'> Trabalho de Circuitos Elétricos - II<h1/>
<h2 align = 'center'> Simulação de Sistemas Elétricos por Quadripolos <h2\>

Este projeto se refere ao estudo sobre dimensionamento de filtros ativos, no qual o mesmo está salvo e disponível para consulta no seguinte [repositório](https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles).

O projeto foi realizado pelos seguinte alunos de Engenharia Elétrica da UFCG:

- Breno Henrique Martins Silva;
- Caíque Moraes de Lima Mangabeira;
- Emerson Matheus Lima da Silva
- Marcos Eduardo Araújo  
- Marcus Vinicius Almeida Filho

---

Utilizando dos conteúdos repassados nas aulas e também nos livros textos que se retratam sobre teoria de quadripolos, iremos utilizar a modelagem de dos componentes da seguinte forma:

### Linha de Transmissão de modelo $\pi$

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/linha.png?raw=true'/></center>

Linhas de transmissão são estruturas físicas utilizadas para transportar energia elétrica de uma fonte (geradores) até os consumidores finais (residências, indústrias, etc.) através de longas distâncias. As principais carácteristicas das Linhas de Transmissão, que estão distribuidas por toda a linha, são a sua Impedância Característica $Z = R + jX_L$ que influência diretamente em como a energia é transmitida e distribuída ao longo da linha, e sua admitância $Y = jX_C$ a admitância é um fator importante que modela o efeito da capacitância e das perdas dielétricas em linhas de transmissão. Seu impacto é notado especialmente em linhas longas, onde contribui para a corrente reativa e pode afetar a regulação de tensão e o fluxo de potência em sistemas de transmissão e distribuição de energia elétrica. 

As Linhas de Transmissão de modelo $\pi$ possuem um comprimento de 80 a 250 quilômetros, e para que possamos trabalhar com este modelo devemos concentrar seus parâmetro em uma equação matricial, no qual a tensão e corrente no final e início da linhas de transmissão possam ser determinadas, assim utilizamos a seguinte matriz para representar o modelo do tipo $\pi$ para linhas de transmissão:

$$
\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 + \frac{ZY}{2} & Z\\ 
Y+\frac{ZY^{2}}{4} & 1 + \frac{ZY}{2}
\end{bmatrix} \cdot \begin{bmatrix}
V_2\\ 
I_2
\end{bmatrix}
$$
---

### Transformador

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/transformador.png?raw=true'/></center>

Iremos utilizar um transformador de um sistema elétrico de potência com o modelo como mostrado acima, e assim como nas Linhas de Transmissão ele também possui uma matriz que representa a relação entre tensão e corrente nos seus terminais, utilizando o conceito de um transformador ideal temos que a sua matriz de transformação é da seguinte forma:

$$
\begin{bmatrix}
n & 0\\
0 & \frac{1}{n}
\end{bmatrix}
$$

Utilizando a matriz de transformação de um Transformador Ideal em conjunto com o modelo de matriz de uma linha de transmissão obteremos a seguinte matriz de transformação:

$$
\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
n(1+YZ_1) &\frac{1}{n}(Z_1+Z_2+YZ_1Z_2) \\ 
nY & \frac{1}{n}(1+YZ_2)
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
I_2
\end{bmatrix}
$$

Em que:

$$ Z_1 = R_1 + jX_{L_1}$$
$$ Z_2 = R_2 + jX_{L_2}$$
$$Y = \frac{R_m + jX_m}{R_m \cdot jX_m}$$

---

### Carga em Série

A tensão e corrente nos terminais de uma carga colocada em série com o circuito é dada por:

$$\begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 &Z \\ 
0 & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
I_2
\end{bmatrix} $$

---

### Carga em Paralelo

A tensão e corrente nos terminais de uma carga colocada em paralelo com o circuito é dada por:

$$ \begin{bmatrix}
V_1\\ 
I_1
\end{bmatrix} = \begin{bmatrix}
1 &0 \\ 
Y & 1
\end{bmatrix}\cdot \begin{bmatrix}
V_2\\ 
I_2
\end{bmatrix}$$

---

In [ ]:
# Importando as Bibliotecas
import numpy as np

In [ ]:
# Declaração de todas as funções:

def MatrixSeriesImpedance(Z):                                   # Retorna a matriz referente a uma impedância em série.
    return np.array([1, Z], [0, 1])

def MarixShuntAdimitance(Y_C):                                  # Retorna a matriz referente a carga em derivação.
    return np.array([1, 0], [Y, 1])

def MatrixTCircuit(Z1, Z2, Y, N, Ym):                           # Retorna a matriz referente ao Transfomador.
    # Calculando a matriz a partir da multiplicação do circuito T com transformador ideal.
    A = N * (1 + (Ym * Z1))
    B = (1/N) * (Z1 + Z2 + (Ym * Z1 * Z2))
    C = N * (Ym)
    D = (1/N) * (1 + (Y * Z2))

    return np.array([A, B], [C, D])

def MatrixRCircuit(Z, Y_L):                                     # Retorna a matriz referente a linha de transmissão.
    A = (1 + ((Z * Y) / 2))
    B = Z
    C = (Y + ((Z * Y**2) / 4))
    D = (1 + ((Z * Y) / 2))

    return np.array([A, B], [C, D])

def MatrizParallelCircuit(Matrix_1, Matriz_2):                  # Retorna a Matriz de associação em pararelo de dois circuitos.
    A1, B1, C1, D1 = Matrix_1[0,0], Matrix_1[0,1], Matrix_1[1,0], Matrix_1[1,1]
    A2, B2, C2, D2 = Matrix_2[0,0], Matrix_2[0,1], Matrix_2[1,0], Matrix_2[1,1]
    # Calculando a matriz para circuitos paralelos.
    A = ((A1 * B2) + (A2 * B1) / (B1 + B2))
    B = ((B1 * B2) / (B1 + B2))
    C = (C1 + C2 + ((A1 - A2) * (D2 - D1) / B1 + B2))
    D = ((B2 * D1) + (B1 * D2) / (B1 + B2))

    return np.array([A, B], [C, D])
    

### Questão 1

<center><img src='https://github.com/brenohmartins/Simulation-of-Electrical-Systems-by-Quadripoles/blob/main/images/SEP.png?raw=true'/></center>

Considerando o sistema elétrico da figura acima, no qual está operando com $60 \text{hz}$, modele o sistema criando as matrizes de transferência $[T]$ a partir dos dados de entrada do modelo que está logo abaixo:

**Impedância de Thévenin**
- $ R_f = 4 \Omega$;
- $ Xf = 0.38 \Omega$. 

**Parâmetros das Linhas de Transmissão**
- $\rho_R = 0.182 \Omega/km$;
- $\rho_L = 2.28 mH/km$;
- $\rho_C = 0.0140 \mu F/km$.

Considere que as $LT_1 , LT_2 , LT_3$ possuem 100 km, a $LT_4$ e $LT_5$ possui 80km e a $LT_6$ possui 120 km.

**Parâmetros comum dos Transformadores**
- $R_1 = 7.6m \Omega$;
- $X_1 = 3.8m \Omega$;
- $R_2 = 33.9m \Omega$;
- $X_2 = 0.85m \Omega$.

**Parâmetros individuais dos Transformadores**
- Para o Transformador 1:
    - $R_{m_1} = 4320 \Omega$ e $X_{m_1} = 5050 \Omega$;
- Para o Transformador 2:
    - $R_{m_2} = 432000 \Omega$ e $X_{m_2} = 505000 \Omega$;
- Para o Transformador 3:
    - $R_{m_3} = 402000 \Omega$ e $X_{m_3} = 607000 \Omega$.

**Cargas em Derivação**

- $R_1 = 84000 \Omega$ e $L_1 = 46H$;
- $R_2 = 1175.55 \Omega$ e $L_2 = 6.43H$;
- $R_3 = 526 \Omega$ e $L_3 = 2.9H$.